In [1]:
from __future__ import division
import os, time
import numpy as np
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import sys
import os
from optparse import OptionParser
import numpy as np
from torch import optim
from PIL import Image
from torch.autograd import Function, Variable
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import Dataset
import cv2
import glob
import pickle
from tqdm import tqdm
import rawpy
%matplotlib inline

input_dir = './Dataset/Sony/short/'
gt_dir = './Dataset/Sony/long/'

result_dir = './results_SWA/'
model_save_path = './results_SWA/net_weights/'
if os.path.isdir(model_save_path) is not True:
    os.makedirs(model_save_path, exist_ok=True)

In [2]:
if torch.cuda.is_available():
    deviceTag = torch.device('cuda')
else:
    deviceTag = torch.device('cpu')
print(deviceTag)

cuda


In [3]:
from utils.swa import SWA

# LOAD DATASET

In [4]:
# get train IDs
train_fns = glob.glob(gt_dir + '0*.ARW')
train_ids = [int(os.path.basename(train_fn)[0:5]) for train_fn in train_fns]

ps = 512  # patch size for training
save_freq = 500

DEBUG = 0
if DEBUG == 1:
    save_freq = 2
    train_ids = train_ids[0:5]


# UNET MODULES

In [5]:
class conv_lrelu(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(conv_lrelu, self).__init__()
        self.conv = nn.Sequential(nn.Conv2d(in_ch,out_ch,3, padding = 1),nn.LeakyReLU())

    def forward(self, x):
        x = self.conv(x)
        return x
    
class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.conv1 = conv_lrelu(in_ch,out_ch)
        self.conv2 = conv_lrelu(out_ch,out_ch)
        self.down =  nn.MaxPool2d((2,2))
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.down(x)
        return x
    

class up(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(up, self).__init__()       
        self.up =  nn.UpsamplingBilinear2d(scale_factor = 2)
        self.conv1 = conv_lrelu(in_ch,out_ch) 
        self.conv2 = conv_lrelu(out_ch,out_ch) 

    def forward(self, x1, x2):
        x1 = self.up(x1)
        x = torch.cat([x2, x1], dim=1)
        x = self.conv1(x)
        x = self.conv2(x)
        return x

class UNet(nn.Module):
    def __init__(self, in_ch = 4, CH_PER_SCALE = [32,64,128,256,512], out_ch = 12):
        super(UNet, self).__init__()
        self.inc = conv_lrelu(in_ch, CH_PER_SCALE[0])
        self.inc2 = conv_lrelu(CH_PER_SCALE[0], CH_PER_SCALE[0])
        self.down1 = down(CH_PER_SCALE[0], CH_PER_SCALE[1]) 
        self.down2 = down(CH_PER_SCALE[1],CH_PER_SCALE[2])
        self.down3 = down(CH_PER_SCALE[2],CH_PER_SCALE[3])                
        self.down4 = down(CH_PER_SCALE[3],CH_PER_SCALE[4])                
        self.up1 = up(CH_PER_SCALE[4]+CH_PER_SCALE[3],CH_PER_SCALE[3])
        self.up2 = up(CH_PER_SCALE[3]+CH_PER_SCALE[2],CH_PER_SCALE[2])
        self.up3 = up(CH_PER_SCALE[2]+CH_PER_SCALE[1],CH_PER_SCALE[1])
        self.up4 = up(CH_PER_SCALE[1]+CH_PER_SCALE[0],CH_PER_SCALE[0])
        self.outc = nn.Conv2d(CH_PER_SCALE[0], out_ch, 1, padding = 0)

    def forward(self, x):
        x0 = self.inc(x)
        x0 = self.inc2(x0)
        x1 = self.down1(x0)
        x2 = self.down2(x1)
        x3 = self.down3(x2)
        x4 = self.down4(x3)
        x3_up = self.up1(x4,x3)
        x2_up = self.up2(x3_up,x2)
        x1_up = self.up3(x2_up,x1)
        out = self.up4(x1_up,x0)
        out = self.outc(out)
#         out = F.pixel_shuffle(out,2) ## Paper final step rearranges 12 channes to 3 RGB channels
#         out = F.hardtanh(out, min_val=0, max_val=1) #Clamp the top and bottom to 0,1 since pixels can only be in this value
        return out
    
class PRIDNet(nn.Module):
    def __init__(self, in_ch = 4, out_ch = 12):
        super(PRIDNet, self).__init__()
        self.feature_extraction = nn.Sequential(conv_lrelu(in_ch, 32), *[conv_lrelu(32, 32) for i in range(3)])
        self.unet0 = UNet(in_ch = 32, out_ch = 12)
        self.unet1 = UNet(in_ch = 32, out_ch = 12)
        self.unet2 = UNet(in_ch = 32, out_ch = 12)
        self.unet3 = UNet(in_ch = 32, out_ch = 12)
        self.unet4 = UNet(in_ch = 32, out_ch = 12)
        self.avgpool1 = nn.AvgPool2d((2,2))
        self.avgpool2 = nn.AvgPool2d((4,4))
        self.avgpool3 = nn.AvgPool2d((8,8))
        self.avgpool4 = nn.AvgPool2d((16,16))
        self.up4 =  nn.UpsamplingBilinear2d(scale_factor = 16)
        self.up3 =  nn.UpsamplingBilinear2d(scale_factor = 8)
        self.up2 =  nn.UpsamplingBilinear2d(scale_factor = 4)
        self.up1 =  nn.UpsamplingBilinear2d(scale_factor = 2)
        self.out =  nn.Conv2d(32+12*5, out_ch, 1, padding = 0)

    def forward(self, x):
        x_feat = self.feature_extraction(x)
        x0 = self.unet0(x_feat)
        x1 = self.up1(self.unet1(self.avgpool1(x_feat)))
        x2 = self.up2(self.unet2(self.avgpool2(x_feat)))
        x3 = self.up3(self.unet3(self.avgpool3(x_feat)))
        x4 = self.up4(self.unet4(self.avgpool4(x_feat)))
        x_unet_all = torch.cat([x_feat,x0,x1,x2,x3,x4], axis = 1)
        out = self.out(x_unet_all)
        
        out = F.pixel_shuffle(out,2) ## Paper final step rearranges 12 channes to 3 RGB channels
        out = F.hardtanh(out, min_val=0, max_val=1) #Clamp the top and bottom to 0,1 since pixels can only be in this value
        return out
    
    def load_my_state_dict(self, state_dict):
        own_state = self.state_dict()
        for name, param in state_dict.items():
            if name not in own_state:
                 continue
            #if isinstance(param, self.Parameter):
            else:
                # backwards compatibility for serialized parameters
                param = param.data
            own_state[name].copy_(param)

# Helper Functions for packing raw and saving images

In [6]:
def pack_raw(raw):
    # pack Bayer image to 4 channels
    im = raw.raw_image_visible.astype(np.float32)
    im = np.maximum(im - 512, 0) / (16383 - 512)  # subtract the black level

    im = np.expand_dims(im, axis=2)
    img_shape = im.shape
    H = img_shape[0]
    W = img_shape[1]

    out = np.concatenate((im[0:H:2, 0:W:2, :],
                          im[0:H:2, 1:W:2, :],
                          im[1:H:2, 1:W:2, :],
                          im[1:H:2, 0:W:2, :]), axis=2)
    return out

In [7]:
# Raw data takes long time to load. Keep them in memory after loaded.
gt_images = [None] * 6000
input_images = {}
input_images['300'] = [None] * len(train_ids)
input_images['250'] = [None] * len(train_ids)
input_images['100'] = [None] * len(train_ids)

g_loss = np.zeros((5000, 1))

allfolders = glob.glob(result_dir + '*0')

for folder in allfolders:
    lastepoch = np.maximum(epochs, int(folder[-4:]))

# Training 

In [8]:
model = PRIDNet()
model = model.cuda()
model = model.train()

In [9]:
def process_img(input_raw_img, model, ratio):
    ## Process image(s) using the given model
    # input_raw_img: numpy array, dimension: (Batch,Height,Width,Channel)
    # ratio: numpy array, dimension: (Batch,)
    model.eval();
    model.to(deviceTag)
    ratio = ratio.reshape(ratio.shape[0],1,1,1)
    input_raw_img = np.transpose(input_raw_img, [0,3,1,2]).astype('float32')*ratio
    input_tensor = torch.from_numpy(input_raw_img.copy()).float().to(deviceTag)
    with torch.no_grad():
        output_tensor = model(input_tensor)
    output_img = output_tensor.cpu().numpy()
    output_img = np.transpose(output_img, [0,2,3,1])
    
    return output_img
    
def validate(model, input_list, gt_list, block_size = None, batch_size = 8):
    assert len(input_list) == len(gt_list)
    
    model.eval();
    PSNR_list = []
    
    for i in range(len(input_list)//batch_size):
        if i%10 == 0:
            print(i)
        input_raw_img_batch = []
        gt_img_batch = []
        ratio_batch = []
        for b in range(batch_size):
            if i*batch_size+b < len(input_list):
                in_path = input_list[i*batch_size+b]
                gt_path = gt_list[i*batch_size+b]
            else:
                break
            in_fn = os.path.basename(in_path)
            gt_fn = os.path.basename(gt_path)
            in_exposure = float(in_fn[9:-5])
            gt_exposure = float(gt_fn[9:-5])
            ratio = min(gt_exposure / in_exposure, 300)
        
            raw = rawpy.imread(in_path)
            input_raw_img = pack_raw(raw)
            
            gt_raw = rawpy.imread(gt_path)
            gt_img = gt_raw.postprocess(use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16)
            gt_img = np.float32(gt_img / 65535.0)
            
            if block_size is not None:
                i_cut, j_cut = np.random.randint(0,input_raw_img.shape[0]-block_size), np.random.randint(0,input_raw_img.shape[1]-block_size)
                gt_img = gt_img[i_cut*2:i_cut*2+block_size*2, j_cut*2:j_cut*2+block_size*2, :]
                input_raw_img = input_raw_img[i_cut:i_cut+block_size, j_cut:j_cut+block_size, :]
            
            ratio_batch.append(ratio)
            input_raw_img_batch.append(input_raw_img)
            gt_img_batch.append(gt_img)
        
        input_raw_img_batch = np.array(input_raw_img_batch)
        ratio_batch = np.array(ratio_batch)
        gt_img_batch = np.array(gt_img_batch)
        
        output_img_batch = process_img(input_raw_img_batch, model, ratio_batch)
        plt.figure()
        plt.imshow(gt_img_batch[0,:,:,:])
        plt.title("Ground Truth")
        plt.figure()
        plt.imshow(output_img_batch[0,:,:,:])
        plt.title("Predicted patch")
        MSE = np.mean((output_img_batch.reshape(output_img_batch.shape[0],-1) - gt_img_batch.reshape(gt_img_batch.shape[0],-1))**2, axis = 1)
        PSNR_batch = 10*np.log10(1/MSE)
        PSNR_list.append(list(PSNR_batch))
    
    Val_PSNR = np.mean(PSNR_list)
        
    return Val_PSNR

In [ ]:
learning_rate = 1e-4
batch_num = 4;
base_opt = optim.SGD(model.parameters(), lr=learning_rate)
optimizer = SWA(base_opt, swa_start=1000, swa_freq=10, swa_lr=0.05)
scheduler = optim.lr_scheduler.StepLR(base_opt, step_size=2000, gamma=0.1) #Step Scheduler. 
criterion = nn.MSELoss()

Start_epoch = 0
epochs = 4000
TrainingLossData = np.zeros(epochs)

for epoch in range(Start_epoch, Start_epoch+epochs):
    print('Starting epoch {}/{}.'.format(epoch + 1, epochs))
    epoch_loss = 0 ## Set Epoch Loss
    count = 0;
    batches_processed = 0
    ##This version has a batch size of 1. In the future conside increasing batchsize
    for ind in np.random.permutation(len(train_ids)):    
        # get the path from image id
        train_id = train_ids[ind]
        in_files = glob.glob(input_dir + '%05d_00*.ARW' % train_id)
        in_path = in_files[np.random.random_integers(0, len(in_files) - 1)]
        in_fn = os.path.basename(in_path)

        gt_files = glob.glob(gt_dir + '%05d_00*.ARW' % train_id)
        gt_path = gt_files[0]
        gt_fn = os.path.basename(gt_path)
        in_exposure = float(in_fn[9:-5])
        gt_exposure = float(gt_fn[9:-5])
        ratio = min(gt_exposure / in_exposure, 300)

        st = time.time()

        if input_images[str(ratio)[0:3]][ind] is None:
            raw = rawpy.imread(in_path)
            input_images[str(ratio)[0:3]][ind] = np.expand_dims(pack_raw(raw), axis=0) * ratio

            gt_raw = rawpy.imread(gt_path)
            im = gt_raw.postprocess(use_camera_wb=True, half_size=False, no_auto_bright=True, output_bps=16)
            gt_images[ind] = np.expand_dims(np.float32(im / 65535.0), axis=0)

        # crop
        H = input_images[str(ratio)[0:3]][ind].shape[1]
        W = input_images[str(ratio)[0:3]][ind].shape[2]

        xx = np.random.randint(0, W - ps)
        yy = np.random.randint(0, H - ps)
        input_patch = input_images[str(ratio)[0:3]][ind][:, yy:yy + ps, xx:xx + ps, :]
        gt_patch = gt_images[ind][:, yy * 2:yy * 2 + ps * 2, xx * 2:xx * 2 + ps * 2, :]

        if np.random.randint(2, size=1)[0] == 1:  # random flip
            input_patch = np.flip(input_patch, axis=1)
            gt_patch = np.flip(gt_patch, axis=1)
        if np.random.randint(2, size=1)[0] == 1:
            input_patch = np.flip(input_patch, axis=2)
            gt_patch = np.flip(gt_patch, axis=2)
        if np.random.randint(2, size=1)[0] == 1:  # random transpose
            input_patch = np.transpose(input_patch, (0, 2, 1, 3))
            gt_patch = np.transpose(gt_patch, (0, 2, 1, 3))
        #(1, 512, 512, 4)
        #(1, 1024, 1024, 3)
        input_patch = np.transpose(input_patch, (0,3,1,2))
        input_patch = torch.from_numpy(input_patch.copy()).cuda()
        gt_patch = np.transpose(gt_patch, (0,3,1,2))
        gt_patch = torch.from_numpy(gt_patch.copy()).cuda()
        ##Batch concatenation
        if count%(batch_num)==0:
            input_patch_all = input_patch
            gt_patch_all = gt_patch
        else:
            input_patch_all = torch.cat([input_patch_all, input_patch], dim=0)
            gt_patch_all = torch.cat([gt_patch_all, gt_patch], dim=0)
        ##Every N batches we ship it back 
        if count%(batch_num)==batch_num-1:
            #print(input_patch_all.shape)
            img_pred = model.forward(input_patch_all)
            loss = criterion(img_pred, gt_patch_all)
            epoch_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            batches_processed += 1
            ##print(epoch_loss/count);
        count = count +1
    scheduler.step()
    print('Epoch finished ! Loss: {}'.format(epoch_loss / batches_processed))
    if epoch == 0:
        trainF= open(result_dir+"TrainingEpoch.txt","w+")
        trainF.write('Epoch,Train_loss\n')
        trainF.write('{},{}\n'.format(epoch, epoch_loss / batches_processed))
        trainF.close()
    else:
        trainF= open(result_dir+"TrainingEpoch.txt","a")
        trainF.write('{},{}\n'.format(epoch, epoch_loss / batches_processed))
        trainF.close()
    TrainingLossData[epoch] = epoch_loss / batches_processed ## Save for plotting
    ################################################ [TODO] ###################################################
    # Perform validation with eval_net() on the validation data
    # Save the model after every 10 epochs. This save our Memory on HPC.
    ##Save Top results after 95%
    if epoch > epochs*0.95:
        torch.save(model.state_dict(),model_save_path + 'sony{}.pth'.format(epoch + 1))
    if epoch%99 == 0:
        torch.save(model.state_dict(),model_save_path + 'sony{}.pth'.format(epoch + 1))

optimizer.swap_swa_sgd()
torch.save(model.state_dict(),model_save_path + 'sony{}.pth'.format(epoch + 1))

Starting epoch 1/4000.


<ipython-input-10-9bd6848c6386>:22: DeprecationWarning: This function is deprecated. Please call randint(0, 0 + 1) instead
  in_path = in_files[np.random.random_integers(0, len(in_files) - 1)]
<ipython-input-10-9bd6848c6386>:22: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  in_path = in_files[np.random.random_integers(0, len(in_files) - 1)]
<ipython-input-10-9bd6848c6386>:22: DeprecationWarning: This function is deprecated. Please call randint(0, 2 + 1) instead
  in_path = in_files[np.random.random_integers(0, len(in_files) - 1)]


Epoch finished ! Loss: 0.08446125553455204
Starting epoch 2/4000.
Epoch finished ! Loss: 0.08514510178938509
Starting epoch 3/4000.
Epoch finished ! Loss: 0.08160317400470377
Starting epoch 4/4000.
Epoch finished ! Loss: 0.08639021832495927
Starting epoch 5/4000.
Epoch finished ! Loss: 0.08197938054800033
Starting epoch 6/4000.
Epoch finished ! Loss: 0.08757441475754604
Starting epoch 7/4000.
Epoch finished ! Loss: 0.08554771190974861
Starting epoch 8/4000.
Epoch finished ! Loss: 0.08134236671030522
Starting epoch 9/4000.
Epoch finished ! Loss: 0.07823340981267393
Starting epoch 10/4000.
Epoch finished ! Loss: 0.08552727324422449
Starting epoch 11/4000.
Epoch finished ! Loss: 0.08355004640761762
Starting epoch 12/4000.
Epoch finished ! Loss: 0.07384061184711754
Starting epoch 13/4000.
Epoch finished ! Loss: 0.07670602169819177
Starting epoch 14/4000.
Epoch finished ! Loss: 0.0822586442809552
Starting epoch 15/4000.
Epoch finished ! Loss: 0.07924061589874327
Starting epoch 16/4000.
Epoc

Epoch finished ! Loss: 0.018764441751409323
Starting epoch 124/4000.
Epoch finished ! Loss: 0.019239833927713335
Starting epoch 125/4000.
Epoch finished ! Loss: 0.0191692448919639
Starting epoch 126/4000.
Epoch finished ! Loss: 0.019322043959982695
Starting epoch 127/4000.
Epoch finished ! Loss: 0.020204559434205294
Starting epoch 128/4000.
Epoch finished ! Loss: 0.01832672767341137
Starting epoch 129/4000.
Epoch finished ! Loss: 0.019791716989129782
Starting epoch 130/4000.
Epoch finished ! Loss: 0.01727230045944452
Starting epoch 131/4000.
Epoch finished ! Loss: 0.019860619597602637
Starting epoch 132/4000.
Epoch finished ! Loss: 0.019500655552837996
Starting epoch 133/4000.
Epoch finished ! Loss: 0.019693822809495033
Starting epoch 134/4000.
Epoch finished ! Loss: 0.01902453196235001
Starting epoch 135/4000.
Epoch finished ! Loss: 0.018019898142665625
Starting epoch 136/4000.
Epoch finished ! Loss: 0.01983831328107044
Starting epoch 137/4000.
Epoch finished ! Loss: 0.020125849952455

Epoch finished ! Loss: 0.018444285925943403
Starting epoch 244/4000.
Epoch finished ! Loss: 0.01871605966007337
Starting epoch 245/4000.
Epoch finished ! Loss: 0.019269630732014775
Starting epoch 246/4000.
Epoch finished ! Loss: 0.017892095958814025
Starting epoch 247/4000.
Epoch finished ! Loss: 0.01817089932737872
Starting epoch 248/4000.
Epoch finished ! Loss: 0.020552623365074397
Starting epoch 249/4000.
Epoch finished ! Loss: 0.018849559500813483
Starting epoch 250/4000.
Epoch finished ! Loss: 0.018419055338017642
Starting epoch 251/4000.
Epoch finished ! Loss: 0.018314186274074017
Starting epoch 252/4000.
Epoch finished ! Loss: 0.019524218165315688
Starting epoch 253/4000.
Epoch finished ! Loss: 0.01863446618663147
Starting epoch 254/4000.
Epoch finished ! Loss: 0.01833842492196709
Starting epoch 255/4000.
Epoch finished ! Loss: 0.019492346327751875
Starting epoch 256/4000.
Epoch finished ! Loss: 0.018941402644850315
Starting epoch 257/4000.
Epoch finished ! Loss: 0.0189248142298

...

Epoch finished ! Loss: 0.014540417597163468
Starting epoch 2145/4000.
Epoch finished ! Loss: 0.0155421162256971
Starting epoch 2146/4000.
Epoch finished ! Loss: 0.016457944782450794
Starting epoch 2147/4000.
Epoch finished ! Loss: 0.014937617257237435
Starting epoch 2148/4000.
Epoch finished ! Loss: 0.014803740952629596
Starting epoch 2149/4000.
Epoch finished ! Loss: 0.015054112358484417
Starting epoch 2150/4000.
Epoch finished ! Loss: 0.015117837185971438
Starting epoch 2151/4000.
Epoch finished ! Loss: 0.014726059522945433
Starting epoch 2152/4000.
Epoch finished ! Loss: 0.015281615406274795
Starting epoch 2153/4000.
Epoch finished ! Loss: 0.015535867249127477
Starting epoch 2154/4000.
Epoch finished ! Loss: 0.015620606066659094
Starting epoch 2155/4000.
Epoch finished ! Loss: 0.016266977495979516
Starting epoch 2156/4000.
Epoch finished ! Loss: 0.015467704110778868
Starting epoch 2157/4000.
Epoch finished ! Loss: 0.015967977629043163
Starting epoch 2158/4000.
Epoch finished ! Loss:

Epoch finished ! Loss: 0.014819207927212119
Starting epoch 2263/4000.
Epoch finished ! Loss: 0.015091919701080769
Starting epoch 2264/4000.
Epoch finished ! Loss: 0.015475128556136041
Starting epoch 2265/4000.
Epoch finished ! Loss: 0.015068517660256475
Starting epoch 2266/4000.
Epoch finished ! Loss: 0.016506251017563044
Starting epoch 2267/4000.
Epoch finished ! Loss: 0.01477559922495857
Starting epoch 2268/4000.
Epoch finished ! Loss: 0.016343126515857877
Starting epoch 2269/4000.
Epoch finished ! Loss: 0.014539964927826077
Starting epoch 2270/4000.
Epoch finished ! Loss: 0.015763612231239676
Starting epoch 2271/4000.
Epoch finished ! Loss: 0.01606588300783187
Starting epoch 2272/4000.
Epoch finished ! Loss: 0.015278850973118097
Starting epoch 2273/4000.
Epoch finished ! Loss: 0.01486476812278852
Starting epoch 2274/4000.
Epoch finished ! Loss: 0.014939465723000468
Starting epoch 2275/4000.
Epoch finished ! Loss: 0.01519077819539234
Starting epoch 2276/4000.
Epoch finished ! Loss: 0

Epoch finished ! Loss: 0.015377101802732796
Starting epoch 2381/4000.
Epoch finished ! Loss: 0.014791631209664046
Starting epoch 2382/4000.
Epoch finished ! Loss: 0.014535409724339842
Starting epoch 2383/4000.
Epoch finished ! Loss: 0.015299486136063934
Starting epoch 2384/4000.
Epoch finished ! Loss: 0.014688832405954599
Starting epoch 2385/4000.
Epoch finished ! Loss: 0.014702156075509265
Starting epoch 2386/4000.
Epoch finished ! Loss: 0.0162797168828547
Starting epoch 2387/4000.
Epoch finished ! Loss: 0.015716116491239517
Starting epoch 2388/4000.
Epoch finished ! Loss: 0.01613048785366118
Starting epoch 2389/4000.
Epoch finished ! Loss: 0.016099852533079682
Starting epoch 2390/4000.
Epoch finished ! Loss: 0.014884499076288193
Starting epoch 2391/4000.
Epoch finished ! Loss: 0.014874507876811549
Starting epoch 2392/4000.
Epoch finished ! Loss: 0.015633704839274287
Starting epoch 2393/4000.
Epoch finished ! Loss: 0.015216003963723778
Starting epoch 2394/4000.
Epoch finished ! Loss: 

Epoch finished ! Loss: 0.015791677858214826
Starting epoch 2499/4000.
Epoch finished ! Loss: 0.015183188673108815
Starting epoch 2500/4000.
Epoch finished ! Loss: 0.015257185627706348
Starting epoch 2501/4000.
Epoch finished ! Loss: 0.015079198242165148
Starting epoch 2502/4000.
Epoch finished ! Loss: 0.015249409072566777
Starting epoch 2503/4000.
Epoch finished ! Loss: 0.014904992748051881
Starting epoch 2504/4000.
Epoch finished ! Loss: 0.01683612884953618
Starting epoch 2505/4000.
Epoch finished ! Loss: 0.014513105060905218
Starting epoch 2506/4000.
Epoch finished ! Loss: 0.01543431639438495
Starting epoch 2507/4000.
Epoch finished ! Loss: 0.01505452535348013
Starting epoch 2508/4000.
Epoch finished ! Loss: 0.014781522809062152
Starting epoch 2509/4000.
Epoch finished ! Loss: 0.0157718924456276
Starting epoch 2510/4000.
Epoch finished ! Loss: 0.015243373322300613
Starting epoch 2511/4000.
Epoch finished ! Loss: 0.015379466174636036
Starting epoch 2512/4000.
Epoch finished ! Loss: 0.

Epoch finished ! Loss: 0.01404972147429362
Starting epoch 2617/4000.
Epoch finished ! Loss: 0.015908778330776842
Starting epoch 2618/4000.
Epoch finished ! Loss: 0.014283477352000773
Starting epoch 2619/4000.
Epoch finished ! Loss: 0.01527911180164665
Starting epoch 2620/4000.
Epoch finished ! Loss: 0.01585986256832257
Starting epoch 2621/4000.
Epoch finished ! Loss: 0.014371758315246553
Starting epoch 2622/4000.
Epoch finished ! Loss: 0.01591038638725877
Starting epoch 2623/4000.
Epoch finished ! Loss: 0.014686023711692541
Starting epoch 2624/4000.
Epoch finished ! Loss: 0.015388512494973838
Starting epoch 2625/4000.
Epoch finished ! Loss: 0.014690470322966576
Starting epoch 2626/4000.
Epoch finished ! Loss: 0.014529076823964714
Starting epoch 2627/4000.
Epoch finished ! Loss: 0.014758236811030657
Starting epoch 2628/4000.
Epoch finished ! Loss: 0.014519248122815043
Starting epoch 2629/4000.
Epoch finished ! Loss: 0.015944201475940646
Starting epoch 2630/4000.
Epoch finished ! Loss: 0

In [ ]:
if os.path.isdir(model_save_path):
            torch.save(model.state_dict(),model_save_path + 'sony{}.pth'.format(epoch + 1))
else:
    os.makedirs(model_save_path, exist_ok=True)
    torch.save(model.state_dict(),model_save_path + 'sony{}.pth'.format(epoch + 1))